## 1. Open Exploartion

In this notebook, we explore and discover the outcomes of the 'base case' scenario; the base case is defined as the 'doing nothing policy' which results in a policy in which all policy levers are set to zero. This exploration is done to explore the uncertainty space and get a quick overview of how our KPI's behave. 

Afterwards, the worst scenario's for our KPI's have been selected and analysed. Following from this analysis, we can define the uncertainty-ranges the model will act most worse in. Later on, these uncertainties will be given extra attention when finding the most robust policies.

For our experiments we will use the latin hypercube sampling (LHS) to generate the points in the parameter space defined by the uncertainties and 'base case' policy. 

In [1]:
# Import libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Import workbench libraries and the model itself
from ema_workbench import (Model, CategoricalParameter, ScalarOutcome, IntegerParameter, RealParameter)
from ema_workbench import (MultiprocessingEvaluator, Policy, Scenario, SequentialEvaluator)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging, utilities
from ema_workbench import save_results
from ema_workbench import load_results

from problem_formulation_new import get_model_for_problem_formulation
from dike_model_function import DikeNetwork 

import time

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [2]:
# # Import case function
# #choose problem formulation number, between 0-5

# # 5 objectives PF
# dike_model, planning_steps = get_model_for_problem_formulation(5)

# def sum_over(*args):
#     return sum(args)

In [7]:
# # Define the base case policy, which sets all the levers that are present within the dike_model to zero.
# # do nothing bolicy
# policies = [Policy("base case", **{lever.name: 0 for lever in dike_model.levers})]

# #Running the experiments (commented out)
# with MultiprocessingEvaluator(dike_model, n_processes = 8) as evaluator:
#      results = evaluator.perform_experiments(scenarios=5000, policies=policies)

# utilities.save_results(results, 'results/5000_scenarios_base_case.csv')
# experiments, outcomes = results

# Load in the results from the CSV we had saved from running the experiments
results = utilities.load_results('results/5000_scenarios_base_case.csv')
experiments, outcomes = results

# We create a 'combined' dataframe, in which the experiments and outcomes are merged within one pandas dataframe.
df_outcomes = pd.DataFrame(outcomes)
combined = pd.concat([experiments,df_outcomes],axis=1,sort=False)

combined.head()

[MainProcess/INFO] results loaded succesfully from C:\Users\ASUS\EPA1361\epa1361_final\results\5000_scenarios_base_case.csv


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_Dike Investment Costs 2,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2
0,82.0,112.588522,10.0,0.060112,207.259283,10.0,0.681905,298.346650,10.0,0.204375,...,0,0.208533,0.000000e+00,0,0.000000,0.0,0,0.0,0.0,0.0
1,14.0,51.029922,1.0,0.277459,302.598285,1.5,0.960055,156.916590,10.0,0.004221,...,0,1.197254,0.000000e+00,0,0.000000,0.0,0,0.0,0.0,0.0
2,128.0,56.198749,1.5,0.880576,136.816986,10.0,0.344347,221.692894,10.0,0.175895,...,0,0.870198,0.000000e+00,0,0.000000,0.0,0,0.0,0.0,0.0
3,96.0,123.505345,1.5,0.797417,322.362756,1.5,0.977793,292.806136,10.0,0.765065,...,0,0.050221,2.064967e+07,0,0.011789,0.0,0,0.0,0.0,0.0
4,52.0,311.328249,10.0,0.433875,130.977495,1.0,0.646960,95.169589,10.0,0.962059,...,0,0.000000,3.580277e+07,0,0.018065,0.0,0,0.0,0.0,0.0


In [ ]:
# The following function is used to aggregate the outcomes into the KPI we want
# This iterates over all the locations and round numbers, and creates a new column summing the values per location and round.
# If we want to aggregate over the location, "aggregate" equals "location" and therefore the KPI is added per location and not in total.
# On the contrary, if aggregate equals "total", the total value is appended to the dataframe.
def aggregate_kpi(data, kpi, aggregate):
    locations = ["A.1", "A.2", "A.3", "A.4", "A.5"]
    kpi_columns = []
    
    if kpi == "RfR Total Costs" or kpi == "Expected Evacuation Costs":
        kpi_columns.append(kpi + " 0")
        kpi_columns.append(kpi + " 1")
        kpi_columns.append(kpi + " 2")
        
        data[kpi] = data[kpi_columns].sum(axis=1)
    else:
        if aggregate is "total":
            for location in locations:
                kpi_columns.append(location + "_" + kpi + " 0")
                kpi_columns.append(location + "_" + kpi + " 1")
                kpi_columns.append(location + "_" + kpi + " 2")

            data[kpi] = data[kpi_columns].sum(axis=1)

        else:
            for location in locations:
                kpi_columns = []
                kpi_columns.append(location + "_" + kpi + " 0")
                kpi_columns.append(location + "_" + kpi + " 1")
                kpi_columns.append(location + "_" + kpi + " 2")

                data[location + "_" + kpi] = data[kpi_columns].sum(axis=1)
                
    return data

# Append the KPIs we would like to analyse. The RfR total costs and expected evacuation costs are included for later on in the analysis, but are not used for the base case analysis, as they are still 0.
combined = aggregate_kpi(combined, "Expected Number of Deaths", "location")
combined = aggregate_kpi(combined, "Expected Number of Deaths", "total")
combined = aggregate_kpi(combined, "Expected Annual Damage", "location")
combined = aggregate_kpi(combined, "Expected Annual Damage", "total")
combined = aggregate_kpi(combined, "RfR Total Costs", "total")
combined = aggregate_kpi(combined, "Expected Evacuation Costs", "total")

In [ ]:
# Define the base case policy, which sets all the levers that are present within the dike_model to zero.
# policy_1
policicy_1 = [Policy("proposed", **{'0_RfR 0':1,
                                  '1_RfR 0':0,
                                  '2_RfR 0':1,
                                  '3_RfR 0':0,
                                  '4_RfR 0':0,
                                  'A.1_DikeIncrease 0':0,
                                  'A.2_DikeIncrease 0':10,
                                  'A.3_DikeIncrease 0':10,
                                  'A.4_DikeIncrease 0':0,
                                  'A.5_DikeIncrease 0':5,
                                  'EWS_DaysToThreat':2})]

#Running the experiments (commented out)
with MultiprocessingEvaluator(dike_model, n_processes = 8) as evaluator:
     results_1 = evaluator.perform_experiments(scenarios=5000, policies=policicy_1 )

utilities.save_results(results_1, 'results/5000_scenarios_proposed.csv')
experiments, outcomes = results

# # Load in the results from the CSV we had saved from running the experiments
# results_1 = utilities.load_results('results/5000_scenarios_base_case.csv')
# experiments, outcomes_1 = results_1

# We create a 'combined' dataframe, in which the experiments and outcomes are merged within one pandas dataframe.
df_outcomes_1 = pd.DataFrame(outcomes_1)
combined_1 = pd.concat([experiments,df_outcomes_1],axis=1,sort=False)

combined_1.head()

In [3]:
# # Define the base case policy, which sets all the levers that are present within the dike_model to zero.


# from ema_workbench import Policy

# policies = [Policy('policy_0', **{'0_RfR 0':0, '0_RfR 1':0, '0_RfR 2':0,
#                                   '1_RfR 0':0, '1_RfR 1':0, '1_RfR 2':0,
#                                   '2_RfR 0':0, '2_RfR 1':0, '2_RfR 2':0,
#                                   '3_RfR 0':0, '3_RfR 1':0, '3_RfR 2':0,
#                                   '4_RfR 0':0, '4_RfR 1':0, '4_RfR 2':0,
#                                   'A.1_DikeIncrease 0':0,
#                                   'A.2_DikeIncrease 0':0,
#                                   'A.3_DikeIncrease 0':0,
#                                   'A.4_DikeIncrease 0':0,
#                                   'A.5_DikeIncrease 0':0,
#                                   'EWS_DaysToThreat':0}),
#            Policy('policy_1',  **{'0_RfR 0':1, '0_RfR 1':1, '0_RfR 2':1,
#                                   '1_RfR 0':0, '1_RfR 1':0, '1_RfR 2':0,
#                                   '2_RfR 0':0, '2_RfR 1':0, '2_RfR 2':0,
#                                   '3_RfR 0':0, '3_RfR 1':0, '3_RfR 2':0,
#                                   '4_RfR 0':1, '4_RfR 1':1, '4_RfR 2':1,
#                                   'A.1_DikeIncrease 0':0,
#                                   'A.2_DikeIncrease 0':10,
#                                   'A.3_DikeIncrease 0':0,
#                                   'A.4_DikeIncrease 0':0,
#                                   'A.5_DikeIncrease 0':0,
#                                   'EWS_DaysToThreat':0}),
#            Policy('policy_2',  **{'0_RfR 0':1, '0_RfR 1':1, '0_RfR 2':1,
#                                   '1_RfR 0':0, '1_RfR 1':0, '1_RfR 2':0,
#                                   '2_RfR 0':1, '2_RfR 1':1, '2_RfR 2':1,
#                                   '3_RfR 0':0, '3_RfR 1':0, '3_RfR 2':0,
#                                   '4_RfR 0':0, '4_RfR 1':0, '4_RfR 2':0,
#                                   'A.1_DikeIncrease 0':0,
#                                   'A.2_DikeIncrease 0':10,
#                                   'A.3_DikeIncrease 0':10,
#                                   'A.4_DikeIncrease 0':0,
#                                   'A.5_DikeIncrease 0':5,
#                                   'EWS_DaysToThreat':2}),
#             Policy('policy_3', **{'0_RfR 0':1, '0_RfR 1':1, '0_RfR 2':1,
#                                   '1_RfR 0':1, '1_RfR 1':1, '1_RfR 2':1,
#                                   '2_RfR 0':1, '2_RfR 1':1, '2_RfR 2':1,
#                                   '3_RfR 0':1, '3_RfR 1':1, '3_RfR 2':1,
#                                   '4_RfR 0':1, '4_RfR 1':1, '4_RfR 2':1,
#                                   'A.1_DikeIncrease 0':10,
#                                   'A.2_DikeIncrease 0':10,
#                                   'A.3_DikeIncrease 0':10,
#                                   'A.4_DikeIncrease 0':10,
#                                   'A.5_DikeIncrease 0':10,
#                                   'EWS_DaysToThreat':4})]

# #Running the experiments (commented out)
# with MultiprocessingEvaluator(dike_model, n_processes = 8) as evaluator:
#      results = evaluator.perform_experiments(scenarios=5, policies=policies, uncertainty_sampling=LHS )


# utilities.save_results(results, 'results/5000ScenariosNoPolicyPF5.csv')
# experiments, outcomes = results

# # Load in the results from the CSV we had saved from running the experiments
# #results = utilities.load_results('results/5000ScenariosNoPolicyPF5.csv')
# #experiments, outcomes = results

# # We create a 'combined' dataframe, in which the experiments and outcomes are merged within one pandas dataframe.
# df_outcomes = pd.DataFrame(outcomes)
# combined = pd.concat([experiments,df_outcomes],axis=1,sort=False)

# combined.head()

In [3]:
from specify import specify_levers
policy0 = Policy('policy_0', **specify_levers(0,0,0,0,0,0,0,0,0,0,0)) # DO NOTHING
policy1 = Policy('policy_1', **specify_levers(0,10,0,0,0,1,0,0,0,1,0)) # DO DIKES at A1, A3
policy2 = Policy('policy_2', **specify_levers(0,10,10,0,5,1,0,1,0,0,2)) # DO DIKES at A1, A2, A3, A5
policy3 = Policy('policy_3', **specify_levers(10,10,10,10,10,1,1,1,1,1,4)) # DO EVERYTHING
n_policies = 4
policies = [policy0, policy1, policy2, policy3]

In [ ]:
# file_name = 'open_exploration_1000_PF6' + '.tar.gz'
# save_results(results, file_name)
# results = load_results(file_name)

In [ ]:
# experiments, outcomes = results
# n_scenarios = len(experiments) / n_policies

In [ ]:
 # Overall
pd.DataFrame(outcomes).describe()

In [ ]:
# # 8 oobjectives PF
# results = load_results('open_exploration_1000_PF6.tar.gz')

In [11]:
# # Define the base case policy, which sets all the levers that are present within the dike_model to zero.
# policies = [Policy("base case", **{lever.name: 0 for lever in dike_model.levers})]

In [9]:
#Running the experiments (commented out)
with SequentialEvaluator(dike_model) as evaluator:
     results = evaluator.perform_experiments(scenarios=1000, policies=4)
#
utilities.save_results(results, 'results/1000_Scenarios_4_Policy_PF5.csv')
experiments, outcomes = results


[MainProcess/INFO] performing 100 scenarios * 4 policies * 1 model(s) = 400 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 240 cases completed
[MainProcess/INFO] 280 cases completed
[MainProcess/INFO] 320 cases completed
[MainProcess/INFO] 360 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] results saved successfully to C:\Users\ASUS\EPA1361\epa1361_final\results\1_Scenarios_4_Policy_PF5.csv


In [15]:
n_scenarios = 1000
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 1 scenarios * 3 policies * 1 model(s) = 3 experiments
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [14]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]

In [10]:
# Load in the results from the CSV we had saved from running the experiments
results = utilities.load_results('results/100_Scenarios_4_Policy_PF5.csv')
experiments, outcomes = results
pd.DataFrame(outcomes).describe()

[MainProcess/INFO] results loaded succesfully from C:\Users\ASUS\EPA1361\epa1361_final\results\100_Scenarios_4_Policy_PF5.csv


In [ ]:
 results = load_results('open_exploration_1000_PF6.tar.gz')

In [13]:
results

(     A.0_ID flood wave shape    A.1_Bmax A.1_Brate  A.1_pfail    A.2_Bmax  \
 0                       56.0  160.858365       1.0   0.469695   62.739555   
 1                        9.0  157.409870       1.5   0.005361   68.258636   
 2                      115.0  146.050957      10.0   0.938728  190.042995   
 3                       34.0  349.520697      10.0   0.521747  313.573847   
 4                       77.0  258.287321       1.0   0.596938  188.262759   
 ..                       ...         ...       ...        ...         ...   
 395                    130.0  295.766905       1.0   0.181821  300.292449   
 396                     54.0  198.949229       1.5   0.552115  100.752744   
 397                     65.0  278.006290       1.0   0.579750  165.869466   
 398                      6.0  211.813457       1.5   0.766477  175.985734   
 399                     82.0  335.618143       1.5   0.371374  286.713771   
 
     A.2_Brate  A.2_pfail    A.3_Bmax A.3_Brate  A.3_pfail  ..

In [22]:
n_scenarios = 1000

experiments, outcomes = perform_experiments(dike_model, scenarios=n_scenarios, uncertainty_sampling=LHS)

utilities.save_outcomes(outcomes, 'results/1000_Scenarios_LHS_PF5.csv')
experiments, outcomes = results

[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 700 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 900 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] experiments finished


AttributeError: module 'ema_workbench.util.utilities' has no attribute 'save_outcomes'

In [28]:
file_name = 'results/1000_Scenarios_LHS_PF5' + '.tar.gz'
save_results(outcomes, file_name)

ValueError: too many values to unpack (expected 2)

In [26]:
utilities.save_outcomes(outcomes, 'results/1000_Scenarios_LHS_PF5.csv')
experiments, outcomes = outcomes

AttributeError: module 'ema_workbench.util.utilities' has no attribute 'save_outcomes'

In [21]:
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS

In [30]:
# Define the base case policy, which sets all the levers that are present within the dike_model to zero.
policies_0 = [Policy("base case", **{lever.name: 0 for lever in dike_model.levers})]

In [31]:
from ema_workbench import SequentialEvaluator

with SequentialEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=1000, policies=policies_0 )

utilities.save_outcomes(results, 'results/1000_Scenarios_base_PF5.csv')
experiments, outcomes = results

[MainProcess/INFO] performing 1000 scenarios * 1 policies * 1 model(s) = 1000 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 700 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 900 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] experiments finished


AttributeError: module 'ema_workbench.util.utilities' has no attribute 'save_outcomes'